In [1]:
import numpy as np

Task 2

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [1]:
!pip install -q roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YDMnnJ9OBWuqL1OBF6Um")
project = rf.workspace("roboflow-gw7yv").project("vehicles-openimages")
dataset = project.version(1).download("yolov7")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 995.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0


Extracting Dataset Version Zip to Vehicles-OpenImages-1 in yolov7pytorch:: 100%|██████████| 2520/2520 [00:00<00:00, 2890.90it/s]


In [154]:
!git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1197 (delta 2), reused 3 (delta 1), pack-reused 1191
Receiving objects: 100% (1197/1197), 74.24 MiB | 30.64 MiB/s, done.
Resolving deltas: 100% (518/518), done.


In [7]:
!pip install -q -r yolov7/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:00


In [4]:
!wget -q https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt


In [157]:
from pathlib import Path

import torch
import random
import numpy as np

from models.yolo import Model
from utils.general import check_requirements, set_logging
from utils.google_utils import attempt_download
from utils.torch_utils import select_device

# dependencies = ['torch', 'yaml']
# check_requirements(Path("/content/yolov7/requirements.txt"), exclude=('pycocotools', 'thop'))
set_logging()


In [158]:
!dir

best.pt				   output_detections.mp4    sample_data       yolov7
compas_video1.mp4		   output_overlay_test.mp4  video1_final.mp4
dave-kim-OBjYlqUq8Mc-unsplash.jpg  output_plot.png	    video3_final.mp4


In [10]:
def custom(path_or_model='path/to/model.pt', autoshape=True):

    model = torch.load(path_or_model, map_location=torch.device('cpu')) if isinstance(path_or_model, str) else path_or_model  # load checkpoint
    if isinstance(model, dict):
        model = model['ema' if model.get('ema') else 'model']  # load model

    hub_model = Model(model.yaml).to(next(model.parameters()).device)  # create
    hub_model.load_state_dict(model.float().state_dict())  # load state_dict
    hub_model.names = model.names  # class names
    if autoshape:
        hub_model = hub_model.autoshape()
    device = select_device('0' if torch.cuda.is_available() else 'cpu')  # default to GPU if available
    return hub_model.to(device)


In [ ]:
# Name of the classes according to class indices.
names= ['Truck', 'Motorcycle', 'Car', 'Bus', 'Ambulance']


# Creating random colors for bounding box visualization.
colors = {
    name: [random.randint(0, 255) for _ in range(3)] for i, name in enumerate(names)
}

In [12]:
model = custom(path_or_model='/content/best.pt')  # custom example

# Verify inference
import numpy as np
from PIL import Image
import cv2

input_img = cv2.imread('/content/dave-kim-OBjYlqUq8Mc-unsplash.jpg')
dim = (640, 640)
# resize image
input_img = cv2.resize(input_img, dim, interpolation = cv2.INTER_AREA)

results = model(input_img)
results.print()
# results.save()
df_prediction = results.pandas().xyxy
df_prediction

Adding autoShape... 
image 1/1: 640x640 3 Cars
Speed: 5.6ms pre-process, 393.8ms inference, 23.9ms NMS per image at shape (1, 3, 640, 640)


[         xmin        ymin        xmax        ymax  confidence  class name
 0  174.165405  272.220337  440.352173  450.955261    0.933980      2  Car
 1  352.403961  244.585968  506.885956  385.862518    0.801100      2  Car
 2  543.683472  241.049469  639.818115  358.438141    0.572014      2  Car]

In [15]:
def plot_detections(frame, df_prediction):

  for i, row in enumerate(df_prediction[0].iterrows()):

    row_index, row_data = row
    (xmin, ymin, xmax, ymax, score, cls_id, cls_name) = row_data.to_list()

    if score < 0.3:
      continue

    start_point = (int(xmin), int(ymin))
    end_point = (int(xmax), int(ymax))

    cv2.rectangle(frame, start_point, end_point, color=(0,255,0), thickness=2)
    cv2.putText(frame, str(cls_name + str(np.round(score, 2))), (start_point[0]-3, start_point[1]-3), cv2.FONT_HERSHEY_SIMPLEX, 0.75, [225, 255, 255], thickness=2,)

  return frame
# from google.colab.patches import cv2_imshow
# cv2_imshow(input_img)


In [ ]:
cap = cv2.VideoCapture('/content/video3_final.mp4')
detections_frames_list = []

if (cap.isOpened()== False):
  print("Error opening video stream or file")

while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    # frames_list.append(frame)
    dim = (640, 640)
    input_img = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA) # resize image

    results = model(input_img)
    results.print()
    # results.save()
    df_prediction = results.pandas().xyxy
    df_prediction

    detections_frames_list.append(plot_detections(frame = input_img, df_prediction= df_prediction))

  else:
    break
cap.release()
# frames = [detections_frames_list[i] for i in range(0,len(detections_frames_list),1)]
len(detections_frames_list)

In [24]:
# import numpy as np
# import cv2 as cv
cap = cv2.VideoCapture("video1_final.mp4")
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))   # float `width`
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
out = cv2.VideoWriter('output_detections.mp4', fourcc, 20.0, (640, 640))
counter=0
for frame in detections_frames_list:
    # ret, frame = cap.read()

    counter+=1
    out.write(frame)
    if cv2.waitKey(1) == ord('q'):
        break
# Release everything if job is finished
print(counter)
cap.release()
out.release()
cv2.destroyAllWindows()

327


Train the model

In [27]:
weapons_ds=[

    "train: ../Vehicles-OpenImages-1/train/images\n",
     "val: ../Vehicles-OpenImages-1/valid/images\n",
     "test: ../Vehicles-OpenImages-1/test/images\n\n",
     "nc: 6 #number of classes\n",
    "names: ['Truck', 'Motorcycle', 'Car', 'Bus', 'Ambulance', 'person']\n"

]

In [28]:
#inside /yolov7
%cd yolov7
with open('data/vehicles_ds.yaml', 'w') as writefile:
  for line in weapons_ds:
    print(line)
    writefile.write(line)

[Errno 2] No such file or directory: 'yolov7'
/content/yolov7
train: ../Vehicles-OpenImages-1/train/images

val: ../Vehicles-OpenImages-1/valid/images

test: ../Vehicles-OpenImages-1/test/images


nc: 6 #number of classes

names: ['Truck', 'Motorcycle', 'Car', 'Bus', 'Ambulance', 'person']



In [25]:
#edit /content/yolov7/cfg/training/yolov7-tiny.yaml file before training

In [29]:
!python train.py --epochs 50 --data data/vehicles_ds.yaml --img 640 640 --cfg cfg/training/yolov7-tiny.yaml --workers 4 --device 0 --batch-size 64 \
--weights '/content/yolov7-tiny.pt' --name yolov7tiny_vehicles_det_fixed_res --hyp data/hyp.scratch.tiny.yaml


2023-12-07 10:09:21.175833: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 10:09:21.175883: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 10:09:21.175981: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 10:09:22.237383: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
YOLOR 🚀 v0.1-128-ga207844 torch 2.1.0+cu118 CUDA:0 (Tesla T4, 15101.8125MB)

Namespace(weights='/content/yolov7-tiny.pt', cfg='cfg/training/yolov7-tiny.yaml', data='data/vehicles_ds.yaml', hyp='data/hyp.scratch.tiny.yaml', epochs=50, batc

In [12]:
%cd yolov7/

/content/yolov7
